# Library

In [1]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import os
import warnings

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Folder Location in LOCAL

In [2]:
os.getcwd()

'C:\\Users\\ramda\\Desktop\\env1\\SCORECARD\\scorecard-build'

# Function

Setiap function yang dibutuhkan dalam proses data cleaning ini akan diletakan pada chunk dibawah

In [3]:
def create_eda_report(data):
    """
    This column need data as dataframe to procces EDA
    Input:
    Dataframe
    return:
    Dataframe
    """
    results = []


    for column in data.columns:
        column_name = column
        column_type = str(data[column].dtype)
        unique_count = len(data[column].unique())
        total = len(data)
        null_count = data[column].isnull().sum()
        non_null_count = data[column].notnull().sum()
        null_percentage = f"{(null_count / total * 100):.2f}%"

        if data[column].dtype == 'object':
            most_frequent = data[column].value_counts().index[0]
            mode_percentage = f"{(data[column].value_counts(normalize=True)[most_frequent] * 100):.2f}%"
            mean_or_top1 = ""
            min_or_bottom = str(data[column].value_counts().index[-1])
            if len(data[column].value_counts()) >= 2:
                min_or_bottom += f": {(data[column].value_counts().values[-1] / total * 100):.2f}%"
            max_or_top1 = str(data[column].value_counts().index[0])
            if len(data[column].value_counts()) >= 2:
                max_or_top1 += f": {(data[column].value_counts().values[0] / total * 100):.2f}%"
        else:
            most_frequent = ""
            mode_percentage = ""
            mean_or_top1 = f"{data[column].mean():.3f}"
            min_or_bottom = data[column].min()
            max_or_top1 = data[column].max()

        results.append({
            'column_name': column_name,
            'type': column_type,
            'unique_count': unique_count,
            'total': total,
            'null_count': null_count,
            'non_null_count': non_null_count,
            'null_percentage': null_percentage,
            'most_frequent': most_frequent,
            'mode_percentage': mode_percentage,
            'mean': mean_or_top1,
            'min_or_bottom': min_or_bottom,
            'max_or_top1': max_or_top1
        })

 
    column_info = pd.DataFrame(results)

    return column_info

# Datasource

Datasource berisi :

- id = id debitur
- limit_bal = Besaran kredit limit yang diberikan dalam dolar NT
- sex = jenis kelamin
  - 1 = laki-laki
  - 2 = perempuan
- education = Pendidikan terakhir
    - 1 = pascasarjana (S2 & S3)
    - 2 = universitas (S1)
    - 3 = high school (SMA)
    - 4 = lain-lain
- marriage = Status pernikahan
    - 1 = menikah
    - 2 = lajang
    - 3 = lainnya
- age = Usia dalam tahun
- pay_* = Status pembayaran dalam bulan April (1) - September (6).
    - 0 = pembayaran tepat waktu
    - 1 = keterlambatan pembayaran satu bulan
    - 2 = keterlambatan pembayaran dua bulan
    - …
    - 8 = keterlambatan pembayaran delapan bulan atau lebih
- bill_amt* = Jumlah tagihan pada bulan April (1) - September (6) dalam dolar NT
- pay_amt* = Jumlah pembayaran/pengeluaran sebelumnya pada bulan April (1) - September(6) dalam dolar NT
- gb_flag = Flagging pembayaran default (gagal bayar) pada bulan berikutnya
    - 1 = default
    - 0 = not default


In [4]:
datasource = pd.read_excel('data/credit_taiwan.xlsx').set_index('id')
datasource.head(3)

,limit_bal,sex,education,marriage,age,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,gb_flag
id,,,,,,,,,,,,,,,,,,,,,,,,
1,20000,2,3,2,39,0,0,2,2,3,2,12241.0,16020.0,16457.0,20906.0,20289.0,20407.0,4000,1000,4750,0,600,0,1
2,100000,1,3,2,49,0,0,0,0,0,0,1440.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
3,80000,2,1,2,26,2,0,0,2,2,2,37097.0,38174.0,40550.0,41577.0,41595.0,43264.0,2000,3000,2000,1000,2500,1000,1


# First Data Observation

In [5]:
datasource.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30952 entries, 1 to 30952
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   limit_bal  30952 non-null  int64  
 1   sex        30952 non-null  int64  
 2   education  30952 non-null  int64  
 3   marriage   30952 non-null  int64  
 4   age        30952 non-null  int64  
 5   pay_1      30952 non-null  int64  
 6   pay_2      30952 non-null  int64  
 7   pay_3      30952 non-null  int64  
 8   pay_4      30952 non-null  int64  
 9   pay_5      30952 non-null  int64  
 10  pay_6      30952 non-null  int64  
 11  bill_amt1  30952 non-null  float64
 12  bill_amt2  30952 non-null  float64
 13  bill_amt3  30952 non-null  float64
 14  bill_amt4  30952 non-null  float64
 15  bill_amt5  30952 non-null  float64
 16  bill_amt6  30952 non-null  float64
 17  pay_amt1   30952 non-null  int64  
 18  pay_amt2   30952 non-null  int64  
 19  pay_amt3   30952 non-null  int64  
 20  pay_amt4   

In [6]:
eda_original = create_eda_report(datasource)
eda_original.to_excel('output/eda_original_data.xlsx', index=False)
eda_original

,column_name,type,unique_count,total,null_count,non_null_count,null_percentage,most_frequent,mode_percentage,mean,min_or_bottom,max_or_top1
0,limit_bal,int64,79,30952,0,30952,0.00%,,,145680.646,10000.0,800000.0
1,sex,int64,2,30952,0,30952,0.00%,,,1.590,1.0,2.0
2,education,int64,4,30952,0,30952,0.00%,,,1.880,1.0,4.0
3,marriage,int64,4,30952,0,30952,0.00%,,,1.552,0.0,3.0
4,age,int64,55,30952,0,30952,0.00%,,,35.321,21.0,75.0
5,pay_1,int64,9,30952,0,30952,0.00%,,,0.562,0.0,8.0
6,pay_2,int64,9,30952,0,30952,0.00%,,,0.637,0.0,8.0
7,pay_3,int64,9,30952,0,30952,0.00%,,,0.568,0.0,8.0
8,pay_4,int64,9,30952,0,30952,0.00%,,,0.469,0.0,8.0
9,pay_5,int64,8,30952,0,30952,0.00%,,,0.412,0.0,8.0


# Data Pre Processing

In [7]:
data_clean = datasource.copy() # copy datasource for easy reload all below in chunk

In [8]:
data_clean[['sex','education','marriage','gb_flag']] = data_clean[['sex','education','marriage','gb_flag']].astype(str)
data_clean['sex'] = data_clean['sex'].replace({'1':'Pria','2':'Wanita'})
data_clean['education'] = data_clean['education'].replace({'1':'Pascasarjana','2':'Sarjana','3':'SMA','4':'Lainya'})
data_clean['marriage'] = data_clean['marriage'].replace({'1':'Menikah','2':'Lajang','3':'Lainya','0': None})

In [9]:
data_clean.head(3)

,limit_bal,sex,education,marriage,age,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,gb_flag
id,,,,,,,,,,,,,,,,,,,,,,,,
1,20000,Wanita,SMA,Lajang,39,0,0,2,2,3,2,12241.0,16020.0,16457.0,20906.0,20289.0,20407.0,4000,1000,4750,0,600,0,1
2,100000,Pria,SMA,Lajang,49,0,0,0,0,0,0,1440.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
3,80000,Wanita,Pascasarjana,Lajang,26,2,0,0,2,2,2,37097.0,38174.0,40550.0,41577.0,41595.0,43264.0,2000,3000,2000,1000,2500,1000,1


In [10]:
eda_final = create_eda_report(data_clean)
eda_final.to_excel('output/eda_data_clean.xlsx',index=False)
eda_final

,column_name,type,unique_count,total,null_count,non_null_count,null_percentage,most_frequent,mode_percentage,mean,min_or_bottom,max_or_top1
0,limit_bal,int64,79,30952,0,30952,0.00%,,,145680.646,10000,800000
1,sex,object,2,30952,0,30952,0.00%,Wanita,59.04%,,Pria: 40.96%,Wanita: 59.04%
2,education,object,4,30952,0,30952,0.00%,Sarjana,48.80%,,Lainya: 1.17%,Sarjana: 48.80%
3,marriage,object,4,30952,52,30900,0.17%,Lajang,53.05%,,Lainya: 1.22%,Lajang: 52.96%
4,age,int64,55,30952,0,30952,0.00%,,,35.321,21,75
5,pay_1,int64,9,30952,0,30952,0.00%,,,0.562,0,8
6,pay_2,int64,9,30952,0,30952,0.00%,,,0.637,0,8
7,pay_3,int64,9,30952,0,30952,0.00%,,,0.568,0,8
8,pay_4,int64,9,30952,0,30952,0.00%,,,0.469,0,8
9,pay_5,int64,8,30952,0,30952,0.00%,,,0.412,0,8


# Create Profile Report

In [11]:
profile = ProfileReport(
    data_clean, 
    title='EDA with YData Profiling',
    explorative= True
)
profile.to_file('output/eda_final.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
data_clean.to_excel('output/data_clean.xlsx')

In [47]:
import pandas as pd

# Contoh matriks korelasi
X_transformed = pd.DataFrame({
    'param1': [1.0, 0.1, 0.1],
    'param2': [0.1, 1.0, 0.1],
    'param3': [0.1, 0.1, 1.0]
}, index=['param1', 'param2', 'param3'])

# Contoh IV
IV = pd.DataFrame({
    'column_name': ['param1', 'param2', 'param3'],
    'iv': [0.05, 0.07, 0.01]
})

# Buat matriks korelasi absolut
corr_matrix = X_transformed.abs()

# List untuk menyimpan kolom yang harus dihapus
cols_to_drop = set()

# Fungsi untuk membandingkan IV dan menentukan kolom yang dihapus
def compare_iv_and_drop(col1, col2):
    iv1 = IV.loc[IV['column_name'] == col1, 'iv'].values[0]
    iv2 = IV.loc[IV['column_name'] == col2, 'iv'].values[0]
    if iv1 < iv2:
        cols_to_drop.add(col1)
    else:
        cols_to_drop.add(col2)

# Menemukan pasangan kolom dengan korelasi lebih dari 0.4 dan memproses sesuai dengan IV
for i in range(len(corr_matrix.columns)):
    col1 = corr_matrix.columns[i]
    if col1 in cols_to_drop:
        continue
    for j in range(i + 1, len(corr_matrix.columns)):
        col2 = corr_matrix.columns[j]
        if col2 in cols_to_drop:
            continue
        if corr_matrix.loc[col1, col2] > 0.4:
            compare_iv_and_drop(col1, col2)
            if col1 in cols_to_drop:
                break

# Filter out correlated columns from IV
IV_filtered = IV[~IV['column_name'].isin(cols_to_drop)]

print("Kolom yang dihapus karena korelasi tinggi dan IV lebih besar:")
print(cols_to_drop)
print("\nDataframe IV setelah filtering:")
print(IV_filtered)


Kolom yang dihapus karena korelasi tinggi dan IV lebih besar:
set()

Dataframe IV setelah filtering:
  column_name    iv
0      param1  0.05
1      param2  0.07
2      param3  0.01


In [39]:
IV

,column_name,iv
0,param1,0.05
1,param2,0.07
2,param3,0.01


In [40]:
X_transformed

,param1,param2,param3
param1,1.0,0.7,0.9
param2,0.7,1.0,0.1
param3,0.9,0.1,1.0
